# Milestone #2: Ravi Raghavan

## Milestone Aim

The goal is to use a **pretrained BERT-Base, Uncased** model and **fine-tune it on the r/Fakeeddit dataset**.

This work presents evaluation results on: 
- Pretrained BERT
- Fine-Tuned BERT


## Script Sanity Check

Please ensure your directory is structured as follows

```text
cleaned_data/
├── test.csv
├── train.csv
└── validation.csv

## Environment Setup

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import torch
import sys
import torch.nn as nn

## Ensure TensorFlow is not used
import os
os.environ["USE_TF"] = "0"

# Import Hugging Face Tooling
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import Trainer, TrainingArguments
import evaluate
from datasets import Dataset

# Define data directory
DATA_DIR = "cleaned_data"

# Define file paths
TRAIN_DATA_FILE = os.path.join(DATA_DIR, "train.csv")
VALIDATION_DATA_FILE = os.path.join(DATA_DIR, "validation.csv")
TEST_DATA_FILE = os.path.join(DATA_DIR, "test.csv")

# For reproducability
random_state = 42

# Use CPU/MPS if possible
device = None
if "google.colab" in sys.modules:
    # Running in Colab
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
else:
    # Not in Colab (e.g., Mac)
    device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

print("Using device:", device)

Using device: mps


## Load Data

In [2]:
TRAIN_DATA = pd.read_csv(TRAIN_DATA_FILE)
VALIDATION_DATA = pd.read_csv(VALIDATION_DATA_FILE)
TEST_DATA = pd.read_csv(TEST_DATA_FILE)

In [3]:
TRAIN_DATA.head()

,clean_title,created_utc,domain,image_url,num_comments,score,subreddit,upvote_ratio,2_way_label,3_way_label,6_way_label
0,this spongebob squarepants branded battery,2019-07-30 20:00:50,i.redd.it,https://preview.redd.it/f39wxxk8yhd31.jpg?widt...,4.0,33,mildlyinteresting,0.95,1,0,0
1,award for careless talk,2011-09-03 17:26:23,i.imgur.com,https://external-preview.redd.it/KgPHCi1u3fY5j...,1.0,14,propagandaposters,1.00,0,1,5
2,four aligned airplanes,2017-11-20 06:05:45,i.redd.it,https://preview.redd.it/88v9axk19phx.jpg?width...,24.0,198,confusing_perspective,0.98,0,2,2
3,columbus discovers the new world,2019-08-28 15:40:17,i.redd.it,https://preview.redd.it/x4wzpd0am7j31.jpg?widt...,5.0,318,fakehistoryporn,0.98,0,2,2
4,feed me drummmmssssssss,2014-05-09 13:23:59,i.imgur.com,https://external-preview.redd.it/yNN57loQnVhLk...,0.0,3,pareidolia,0.62,0,2,2


In [4]:
VALIDATION_DATA.head()

,clean_title,created_utc,domain,image_url,num_comments,score,subreddit,upvote_ratio,2_way_label,3_way_label,6_way_label
0,driving toward the great pyramid of giza,2015-08-20 00:44:04,i.imgur.com,https://external-preview.redd.it/qOoqG_Bzmxrxx...,0.0,5,misleadingthumbnails,0.65,0,2,2
1,how to get charizard and other starters in pok...,2016-09-04 05:55:37,inverse.com,https://external-preview.redd.it/2HRiWyzJP1Obr...,1.0,105,savedyouaclick,0.94,0,2,5
2,my friend gave me this euro with this guy on i...,2019-05-13 14:23:20,i.redd.it,https://preview.redd.it/hty90t90nzx21.jpg?widt...,1.0,38,mildlyinteresting,0.87,1,0,0
3,ho chi minh visits the united states,2018-06-16 05:21:17,i.redd.it,https://preview.redd.it/iepetf7ksa411.jpg?widt...,0.0,27,fakehistoryporn,0.92,0,2,2
4,bipartisan bill seeking deeper russiaputin pro...,2019-03-14 04:36:29,northcountrypublicradio.org,https://external-preview.redd.it/YObKn9FcqRgNa...,1.0,18,usnews,0.96,1,0,0


In [5]:
TEST_DATA.head()

,clean_title,created_utc,domain,image_url,num_comments,score,subreddit,upvote_ratio,2_way_label,3_way_label,6_way_label
0,us antiterror center helped police track envir...,2019-10-02 10:48:11,theguardian.com,https://external-preview.redd.it/D1BJCo8ZObamg...,1.0,26,usnews,1.00,1,0,0
1,comey what can i say im just a catty bitch fro...,2018-04-16 17:02:44,politics.theonion.com,https://external-preview.redd.it/JDKJYZJX-LDW9...,1.0,45,theonion,0.94,0,2,1
2,watch mom attacked over shopping cart in wi wa...,2018-07-29 04:37:30,wtvm.com,https://external-preview.redd.it/wRxI1DzMEAg-8...,1.0,3,usnews,1.00,1,0,0
3,a different kind of uplifting news maryland ac...,2018-08-15 03:11:45,espn.com,https://external-preview.redd.it/VaZiSdjw-4kF_...,2.0,13,upliftingnews,0.88,1,0,0
4,this weimaraner at doggy care,2018-09-14 20:20:20,i.redd.it,https://preview.redd.it/4iwrppt0j9m11.jpg?widt...,5.0,13,photoshopbattles,0.90,1,0,0


## Compute Class Proportions

In [6]:
# Compute Class Proportions
p0 = (TRAIN_DATA['2_way_label'] == 0).mean() # Computes the percentage of our training dataset that has label = 0 [Fake News]
p1 = (TRAIN_DATA['2_way_label'] == 1).mean() # Computes the percentage of our training dataset that has label = 1 [Non-Fake News]
print(f"{p0  * 100}% of our dataset has label = 0 and {p1  * 100}% of our dataset has label = 1")

44.664506061697274% of our dataset has label = 0 and 55.335493938302726% of our dataset has label = 1


## Define Prior Adjusted Loss Criterion

In [7]:
# Define Weighted Loss Criterion
class_weights = torch.tensor([p1, p0]).float().to(device)
custom_criterion = nn.CrossEntropyLoss(weight = class_weights)
print(f"Class Weights: {class_weights}")

Class Weights: tensor([0.5534, 0.4466], device='mps:0')


## Fetch BERT From HuggingFace

In [8]:
# Fetch BERT Model from HuggingFace
bert_model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
model = BertForSequenceClassification.from_pretrained(bert_model_name, num_labels = 2) # num_labels = 2 since we have 2 classes!

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Create `Hugging Face` Datasets [Train + Dev + Test]


In [9]:
train_hf_dataset = Dataset.from_pandas(TRAIN_DATA)
dev_hf_dataset = Dataset.from_pandas(VALIDATION_DATA)
test_hf_dataset = Dataset.from_pandas(TEST_DATA)

## Tokenize Text Data

In [10]:
def tokenize_function(row):
  tokens = tokenizer(row['clean_title'], truncation = True, padding = 'max_length', max_length = tokenizer.model_max_length)
  row['input_ids'] = tokens['input_ids']
  row['attention_mask'] = tokens['attention_mask']
  row['token_type_ids'] = tokens['token_type_ids']
  row['label'] = int(row['2_way_label'])
  return row

In [11]:
train_hf_dataset = train_hf_dataset.map(tokenize_function)
dev_hf_dataset = dev_hf_dataset.map(tokenize_function)
test_hf_dataset = test_hf_dataset.map(tokenize_function)

Map:   0%|          | 0/33324 [00:00<?, ? examples/s]

Map:   0%|          | 0/33316 [00:00<?, ? examples/s]

Map:   0%|          | 0/33519 [00:00<?, ? examples/s]

## Define Accuracy, Precision, Recall, and F1 Metrics from Hugging Face

In [12]:
accuracy_metric = evaluate.load("accuracy")
precision_metric = evaluate.load("precision")
recall_metric = evaluate.load('recall')
f1_metric = evaluate.load("f1")

## Define a compute_metrics function

In [13]:
def compute_metrics(eval_pred):
    # Get the model predictions
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Return Metrics
    return {
        "accuracy": accuracy_metric.compute(predictions=predictions, references=labels)['accuracy'], # Accuracy
        "pos_precision": precision_metric.compute(predictions=predictions, references=labels, pos_label = 1, average = 'binary', zero_division = 0)["precision"], # Precision on the Class w/ Label = 1 [Hate Samples]
        "pos_recall": recall_metric.compute(predictions=predictions, references=labels, pos_label = 1, average = 'binary', zero_division = 0)['recall'], # Recall on the Class w/ Label = 1 [Hate Samples]
        "pos_f1": f1_metric.compute(predictions=predictions, references=labels, pos_label = 1, average = 'binary')["f1"], # F1 Score on the Class w/ Label = 1 [Hate Samples]
        "neg_precision": precision_metric.compute(predictions=predictions, references=labels, pos_label = 0, average = 'binary', zero_division = 0)['precision'], # Precision on the Class w/ Label = 0 [Non-Hate Samples]
        "neg_recall": recall_metric.compute(predictions=predictions, references=labels, pos_label = 0, average = 'binary', zero_division = 0)['recall'], # Recall on the Class w/ Label = 0 [Non-Hate Samples]
        "neg_f1": f1_metric.compute(predictions=predictions, references=labels, pos_label = 0, average = 'binary')['f1'], # F1 Score on the Class w/ Label = 0 [Non-Hate Samples]
        "f1_macro": f1_metric.compute(predictions=predictions, references=labels, average='macro')['f1'], # Macro F1 Score
        "f1_micro": f1_metric.compute(predictions=predictions, references=labels, average='micro')['f1'], # Micro F1 Score
        "f1_weighted": f1_metric.compute(predictions=predictions, references=labels, average='weighted')['f1'], # Weighted F1 Score
    }


## Subclass the `Trainer` Class from HuggingFace to use Custom Loss Criterion


In [14]:
# Create a subclassed Trainer that enables us to use the custom loss function defined earlier
class SubTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs = False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss = custom_criterion(logits, labels)
        return (loss, outputs) if return_outputs else loss

## **Initialize the `TrainingArguments` and `Trainer`**

In [15]:
training_args = TrainingArguments(
    output_dir="Milestone2-Baseline-BERT-FineTuning",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    learning_rate=5e-5,
    num_train_epochs=3,
    save_strategy="steps",      # save checkpoints every N steps
    save_steps=100,             # save every 100 steps
    eval_strategy="steps",      # evaluate every N steps
    eval_steps=100,             # evaluate every 100 steps
    logging_strategy="steps",
    logging_steps=100,          # log every 100 steps
    report_to="none",
    full_determinism=True
)

trainer = SubTrainer(
    model=model,
    args=training_args,
    train_dataset=train_hf_dataset,
    eval_dataset=dev_hf_dataset,
    compute_metrics=compute_metrics,
)

# **Evaluate Pre-Trained Model on Train, Dev, and Test Datasets**

In [16]:
# Split: Train, Dev, or Test
def generate_evaluation_results(split):
    dataset = None
    if split == "train":
        dataset = train_hf_dataset
    elif split == "dev" or split == "validation" or split == "val":
        dataset = dev_hf_dataset
    elif split == "test":
        dataset = test_hf_dataset
    
    results = trainer.evaluate(eval_dataset=dataset, metric_key_prefix=split)
    df_results = pd.DataFrame([results])
    df_results.to_csv(f"Milestone #2 Pre-Trained BERT Baseline {split} Results.csv", index=False)
    print(f"Saved {split} evaluation metrics to Milestone #2 Pre-Trained BERT Baseline {split} Results.csv")

# Generate Evaluation Results on Train, Dev, and Test Splits
generate_evaluation_results("train")
generate_evaluation_results("dev")
generate_evaluation_results("test")

/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Saved train evaluation metrics to Milestone #2 Pre-Trained BERT Baseline train Results.csv


/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Saved dev evaluation metrics to Milestone #2 Pre-Trained BERT Baseline dev Results.csv


/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Saved test evaluation metrics to Milestone #2 Pre-Trained BERT Baseline test Results.csv


# **Train the Model: `Fine-Tuning`**

In [17]:
trainer.train() # Always Resume from Last Checkpoint to Save Time
trainer.save_model('Milestone2-Baseline-BERT-FinalModel') # Save the Final Model
trainer.save_state() # Save the State of the Trainer (e.g. Losses, etc)

/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss,Validation Loss,Model Preparation Time,Accuracy,Pos Precision,Pos Recall,Pos F1,Neg Precision,Neg Recall,Neg F1,F1 Macro,F1 Micro,F1 Weighted
100,0.578300,0.496525,0.001000,0.768129,0.767712,0.833947,0.799460,0.768761,0.686305,0.725197,0.762328,0.768129,0.766354
200,0.507000,0.492597,0.001000,0.791962,0.784246,0.861677,0.821140,0.803976,0.705292,0.751408,0.786274,0.791962,0.790054
300,0.485400,0.445102,0.001000,0.798115,0.824936,0.806976,0.815857,0.766356,0.787099,0.776589,0.796223,0.798115,0.798352
400,0.464500,0.441631,0.001000,0.792352,0.853781,0.754549,0.801104,0.733380,0.839348,0.782794,0.791949,0.792352,0.792942
500,0.468300,0.421180,0.001000,0.812973,0.837984,0.821328,0.829572,0.783231,0.802586,0.792790,0.811181,0.812973,0.813175
600,0.429000,0.412693,0.001000,0.815734,0.854962,0.803889,0.828639,0.773049,0.830461,0.800727,0.814683,0.815734,0.816196
700,0.428700,0.405913,0.001000,0.816334,0.863537,0.794086,0.827357,0.767277,0.843994,0.803809,0.815583,0.816334,0.816859
800,0.417100,0.400847,0.001000,0.818556,0.861838,0.801018,0.830316,0.772578,0.840358,0.805044,0.817680,0.818556,0.819050
900,0.418800,0.397422,0.001000,0.823628,0.836542,0.847325,0.841899,0.807103,0.794169,0.800584,0.821241,0.823628,0.823481
1000,0.406900,0.409410,0.001000,0.811832,0.873278,0.772584,0.819851,0.752724,0.860625,0.803066,0.811459,0.811832,0.812368


/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning:

# **Evaluate Fine-Tuned Model on Train, Dev, and Test Datasets**

In [18]:
# Split: Train, Dev, or Test
def generate_evaluation_results(split):
    dataset = None
    if split == "train":
        dataset = train_hf_dataset
    elif split == "dev" or split == "validation" or split == "val":
        dataset = dev_hf_dataset
    elif split == "test":
        dataset = test_hf_dataset
    
    results = trainer.evaluate(eval_dataset=dataset, metric_key_prefix=split)
    df_results = pd.DataFrame([results])
    df_results.to_csv(f"Milestone #2 Fine-Tuned BERT Baseline {split} Results.csv", index=False)
    print(f"Saved {split} evaluation metrics to Milestone #2 Fine-Tuned BERT Baseline {split} Results.csv")

# Generate Evaluation Results on Train, Dev, and Test Splits
generate_evaluation_results("train")
generate_evaluation_results("dev")
generate_evaluation_results("test")

/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Saved train evaluation metrics to Milestone #2 Fine-Tuned BERT Baseline train Results.csv


/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Saved dev evaluation metrics to Milestone #2 Fine-Tuned BERT Baseline dev Results.csv


/opt/anaconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Saved test evaluation metrics to Milestone #2 Fine-Tuned BERT Baseline test Results.csv
